In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [167]:
social = pd.read_csv('degree.csv',encoding='utf-8')
social1 = pd.read_csv('degree1.csv',encoding='utf-8')
social.head()

,Vertices,Degree,Clustering,Class
0,n10000035,3,0.0,1
1,n10000055,1,0.0,0
2,n10000063,1,0.0,0
3,n10007006,1,0.0,0
4,n10000064,3,0.0,1


In [168]:
social.drop(['Vertices'],axis = 1,inplace = True)
social1.drop(['Vertices'],axis = 1,inplace = True)

In [169]:
print(social.describe())
total_sample=social.shape[0]
print(total_sample)

              Degree     Clustering          Class
count  548562.000000  548562.000000  548562.000000
mean        2.031362       0.019425       0.365426
std        22.142871       0.089878       0.736318
min         1.000000       0.000000       0.000000
25%         1.000000       0.000000       0.000000
50%         1.000000       0.000000       0.000000
75%         2.000000       0.000000       1.000000
max      7015.000000       1.000000       4.000000
548562


In [170]:
X = social.iloc[:,0:2].values
Y = social['Class'].values
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =1/3)
X_traino=X_train
X_testo=X_test
Y_traino=Y_train
Y_testo=Y_test

In [171]:
X_train.shape

(365708, 2)

In [172]:
X_test.shape

(182854, 2)

In [173]:
stdscaler = StandardScaler()
X_train = stdscaler.fit_transform(X_train)
X_test = stdscaler.transform(X_test)

In [174]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,Y_train)

GaussianNB()

In [175]:
y_pred = clf.predict(X_test)
y_predproba=clf.predict_proba(X_test)
y_predprobatrain=clf.predict_proba(X_train)

In [176]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(Y_test, y_pred)

In [177]:
print(cm)

[[133633      0      0      0      0]
 [     0  39314      0      0      0]
 [     0      0   5166      0      1]
 [     0      0      0   1662      0]
 [     0      0      0      0   3078]]


In [178]:
evidence=[]
sample=[]
for i in range(4):
    dfx=social[social.Class==(i+1)]
    sample.append(dfx.shape[0])
    evidence.append(dfx.shape[0]/total_sample)


In [183]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,Y_train)
y_pred3=knn.predict(X_test)

In [184]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm3 = confusion_matrix(Y_test, y_pred3)
print(cm3)

[[133633      0      0      0      0]
 [     0  39312      1      0      1]
 [     0      0   5164      2      1]
 [     0      0      0   1655      7]
 [     0      0      0      1   3077]]


In [190]:
import sklearn.neural_network 
neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(4, ), alpha=0.2)
neural_network.fit(X_train, Y_train) 
y_pred10 = neural_network.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
cm10 = confusion_matrix(Y_test, y_pred10)
print(cm10)

[[133633      0      0      0      0]
 [     0  39314      0      0      0]
 [     0      0   5167      0      0]
 [     0      0   1507    127     28]
 [     0      0      0      0   3078]]


In [179]:
import openpyxl
wb=openpyxl.Workbook()
s1=wb.active
s1['A1']='degree'
s1['B1']='clustering'
s1['C1']='ZP'
s1['D1']='YP'
wb.save('output51.xlsx')

In [182]:
traintotal=X_traino.shape[0]
yp=[]
xp=[]
zp=[]
workbook=openpyxl.load_workbook('output51.xlsx')
sheet=workbook.active
for i in range(traintotal):
    if(Y_traino[i]==0):
        yp.append(y_predprobatrain[i,0])
        xp.append(X_traino[i,:])

testtotal=X_testo.shape[0]

for i in range(testtotal):
    if(Y_testo[i]==0):
        yp.append(y_predproba[i,0])
        xp.append(X_testo[i,:])

for k in range(sample[0]):
    zp.append(yp[k]*evidence[0])


zsup=max(zp)

for k in range(sample[0]):
    zp[k]=zp[k]/zsup

j=2
for k in range(sample[0]):
    sheet.cell(j,1).value=xp[k][0]
    sheet.cell(j,2).value=xp[k][1]
    sheet.cell(j,3).value=zp[k]
    sheet.cell(j,4).value=yp[k]
    j+=1
workbook.save('output51.xlsx')